In [28]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [29]:
df = pd.read_csv("/content/drive/MyDrive/comsumer_temperature/datasets/data.csv", encoding="shift-jis")
df.head()

,,,ダウンロードした時刻：2023/10/13 13:42:24
NaN,つくば（館野）,つくば（館野）,つくば（館野）
年月日,平均気温(℃),平均気温(℃),平均気温(℃)
NaN,NaN,品質情報,均質番号
2003/1/1,2.9,8,1
2003/1/2,1.6,8,1


In [30]:
df = df.reset_index()
df = df.drop([0, 2], axis=0).reset_index()
df.head()

,index,level_0,level_1,level_2,ダウンロードした時刻：2023/10/13 13:42:24
0,1,年月日,平均気温(℃),平均気温(℃),平均気温(℃)
1,3,2003/1/1,2.9,8,1
2,4,2003/1/2,1.6,8,1
3,5,2003/1/3,-0.8,8,1
4,6,2003/1/4,4.0,8,1


In [31]:
df = df[["level_0", "level_1"]]
df.columns = df.iloc[0, :]

In [32]:
df = df.drop([0], axis=0).reset_index(drop=True)
df

,年月日,平均気温(℃)
0,2003/1/1,2.9
1,2003/1/2,1.6
2,2003/1/3,-0.8
3,2003/1/4,4.0
4,2003/1/5,2.3
...,...,...
7585,2023/10/8,15.9
7586,2023/10/9,16.0
7587,2023/10/10,18.5
7588,2023/10/11,17.0


In [33]:
df.to_csv("/content/drive/MyDrive/comsumer_temperature/datasets/temprature_1.csv")

In [34]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7590 entries, 0 to 7589
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   年月日      7590 non-null   object
 1   平均気温(℃)  7590 non-null   object
dtypes: object(2)
memory usage: 118.7+ KB


In [35]:
df.isna().sum()

0
年月日        0
平均気温(℃)    0
dtype: int64

In [36]:
df["年月日"] = pd.to_datetime(df["年月日"])
df["平均気温(℃)"] = pd.to_numeric(df["平均気温(℃)"])
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7590 entries, 0 to 7589
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   年月日      7590 non-null   datetime64[ns]
 1   平均気温(℃)  7590 non-null   float64       
dtypes: datetime64[ns](1), float64(1)
memory usage: 118.7 KB


In [37]:
df["POSIX"] = df['年月日'].astype(np.int64).values // 10**9
df.head()

,年月日,平均気温(℃),POSIX
0,2003-01-01,2.9,1041379200
1,2003-01-02,1.6,1041465600
2,2003-01-03,-0.8,1041552000
3,2003-01-04,4.0,1041638400
4,2003-01-05,2.3,1041724800


In [38]:
df["year"] = df["年月日"].dt.year
df["month"] = df["年月日"].dt.month
df["day"] = df["年月日"].dt.day
df["dayofweek"] = df["年月日"].dt.dayofweek
df.head()

,年月日,平均気温(℃),POSIX,year,month,day,dayofweek
0,2003-01-01,2.9,1041379200,2003,1,1,2
1,2003-01-02,1.6,1041465600,2003,1,2,3
2,2003-01-03,-0.8,1041552000,2003,1,3,4
3,2003-01-04,4.0,1041638400,2003,1,4,5
4,2003-01-05,2.3,1041724800,2003,1,5,6


In [39]:
day = 15

for i in range(1, day):
    df[f"{i}dayago"] = df["平均気温(℃)"].shift(i)

In [40]:
df.shape

(7590, 21)

In [41]:
N = len(df["POSIX"])
N_train = round(N * .8)

In [42]:
df.to_csv("/content/drive/MyDrive/comsumer_temperature/datasets/temprature_2.csv")

In [43]:
df = df.dropna(axis=0)

In [44]:
df.shape

(7576, 21)

In [45]:
df.to_csv("/content/drive/MyDrive/comsumer_temperature/datasets/temprature_3.csv")

In [ ]:
X = df.drop(["平均気温(℃)", "年月日"], axis=1).values
y = df["平均気温(℃)"].values

In [ ]:
X_train, X_test = X[:N_train], X[N_train:]
y_train, y_test = y[:N_train], y[N_train:]

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error

In [ ]:
rf = RandomForestRegressor(n_estimators=100,
                           random_state=0,
                           )
rf.fit(X_train, y_train)

In [ ]:
y_pred = rf.predict(X_test)

In [ ]:
plt.figure(figsize=(18, 10))
plt.plot(X, y, color="gray", linestyle="dashdot")
plt.plot(X_test, y_pred, "+", color="red")
plt.xlim([1.0e9, 1.75e9])
plt.grid()

plt.show()

In [ ]:
mse = mean_squared_error(y_test, y_pred)
print(f"MSE: {mse:.3f}")

In [ ]:
mae = mean_absolute_error(y_test, y_pred)
print(f"MAE: {mae:.3f}")

In [ ]:
from sklearn.metrics import r2_score

print(f"Train R2: {rf.score(X_train, y_train):2f}")
print(f"Test R2: {r2_score(y_test, y_pred):2f}")